In [1]:
import pandas as pd
from pandas import Series, DataFrame
import string
import country_converter as coco
import os

In [2]:
# https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy.html
# Statistical Review of World Energy - all data, 1965-2021
# https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/xlsx/energy-economics/statistical-review/bp-stats-review-2022-all-data.xlsx 


In [3]:
path = '/Users/ariana/desktop/historical_tech'
os.chdir(path)

In [4]:
def read_bp(sheet):
    df = pd.read_excel('raw data/bp-stats-review-2022-all-data.xlsx', sheet_name=sheet, header=2, 
                       index_col=0, na_values=['-', '^','♦'])
    omit = []
    for col in df.columns:
        if type(col) != int:
            omit.append(col)
    df.drop(columns=omit, inplace=True)
    df.dropna(how='all', inplace=True)
    idx = []
    for country in df.index:
        x = country.strip(string.digits)
        idx.append(x)
    df['Country Name'] = idx
    unit = df.index.name.rstrip('*')
    unit = unit.strip(string.digits)
    df['Unit'] = unit
    df['Data Source'] = 'BP'
    spatial_scale = []
    for country in df.index:
        strings = ['Total', 'Rest of World', 'Other', 'European Union', 'OECD', 'Central America', \
                   'Eastern Africa', 'Middle Africa', 'Western Africa', 'OPEC']
        for s in strings:
            if s in country and country != 'Total World':
                df.drop(country, inplace=True)
    iso2 = []
    for country in df.index:
        if country=='USSR':
            iso2.append('SU')
            spatial_scale.append('National')
        elif country=='Netherlands Antilles':
            iso2.append('AN')
            spatial_scale.append('National')
        elif country=='Total World':
            iso2.append('World')
            spatial_scale.append('Global')
        else:
            iso2.append(coco.convert(names=country, to='iso2'))
            spatial_scale.append('National')
    df.replace('Total World', 'World', inplace=True)
    df['Spatial Scale'] = spatial_scale
    df['Country Code'] = iso2
    df.reset_index(drop=True, inplace=True)
    df.replace({'Petajoules':'petajoules','Million tonnes':'million metric tons',
                'Million tonnes ':'million metric tons','million tonnes ':'million metric tons',
                'Thousand tonnes':'thousand metric tons','Tonnes':'metric tons',
                'Terawatt-hours':'TWh','Billion cubic metres':'billion cubic metres',
               'Thousand barrels daily':'thousand barrels/day'}, inplace=True)
    return df

In [5]:
def tech_name(df, tech):
    df['Technology Name'] = tech
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    df.set_index('ID', inplace=True)
    return df

In [6]:
bp = []

In [7]:
bp_oil_production = read_bp('Oil Production - Tonnes')
bp_oil_production['Metric'] = 'Annual Production'
bp_oil_production = tech_name(bp_oil_production, 'Oil Production')
bp.append(bp_oil_production)

In [8]:
bp_oil_refining = read_bp('Oil - Refining capacity')
bp_oil_refining['Metric'] = 'Total Capacity'
bp_oil_refining = tech_name(bp_oil_refining, 'Oil Refining Capacity')
bp.append(bp_oil_refining)

In [9]:
bp_gas = read_bp('Gas Production - Bcm')
bp_gas['Metric'] = 'Annual Production'
bp_gas = tech_name(bp_gas, 'Natural Gas Production')
bp.append(bp_gas)

In [10]:
bp_coal = read_bp('Coal Production - Tonnes')
bp_coal['Metric'] = 'Annual Production'
bp_coal = tech_name(bp_coal, 'Coal Production')
bp.append(bp_coal)

In [11]:
bp_nuclear = read_bp('Nuclear Generation - TWh')
bp_nuclear['Metric'] = 'Annual Production'
bp_nuclear = tech_name(bp_nuclear, 'Nuclear Energy')
bp.append(bp_nuclear)

In [12]:
bp_hydro = read_bp('Hydro Generation - TWh')
bp_hydro['Metric'] = 'Annual Production'
bp_hydro = tech_name(bp_hydro, 'Hydroelectricity')
bp.append(bp_hydro)

In [13]:
bp_renewable = read_bp('Renewable power - TWh')
bp_renewable['Metric'] = 'Annual Production'
bp_renewable = tech_name(bp_renewable, 'Renewable Power')
bp.append(bp_renewable)

In [14]:
bp_electricity_gen = read_bp('Electricity Generation')
bp_electricity_gen['Metric'] = 'Annual Production'
bp_electricity_gen = tech_name(bp_electricity_gen, 'Electricity')
bp.append(bp_electricity_gen)

In [15]:
# bp_lithium = read_bp('Lithium Production-Reserves')
# bp_lithium['Metric'] = 'Annual Production'
# bp_lithium = tech_name(bp_lithium, 'Lithium Mine Production')
# bp.append(bp_lithium)

In [16]:
# bp_cobalt = read_bp('Cobalt Production-Reserves')
# bp_cobalt['Metric'] = 'Annual Production'
# bp_cobalt = tech_name(bp_cobalt, 'Cobalt Mine Production')
# bp.append(bp_cobalt)

In [17]:
# bp_rare_earth = read_bp('Rare Earth Production-Reserves')
# bp_rare_earth['Metric'] = 'Annual Production'
# bp_rare_earth = tech_name(bp_rare_earth, 'Rare Earth Mine Production')
# bp.append(bp_rare_earth)

In [18]:
# bp_graphite = read_bp('Graphite Production-Reserves')
# bp_graphite['Metric'] = 'Annual Production'
# bp_graphite = tech_name(bp_graphite, 'Graphite Mine Production')
# bp.append(bp_graphite)

In [19]:
def read_bp2(sheet):
    missing_values = ['-', '^','♦']
    df = pd.read_excel('raw data/bp-stats-review-2022-all-data.xlsx', sheet_name=sheet, header=2, index_col=0, 
                       na_values=missing_values, skipfooter=40)
    omit = []
    for col in df.columns:
        if type(col) != int:
            omit.append(col)
    df.drop(columns=omit, inplace=True)
    df.dropna(how='all', inplace=True)
    idx = []
    for country in df.index:
        x = country.strip(string.digits)
        idx.append(x)
    df['Country Name'] = idx
    unit = df.index.name.rstrip('*')
    unit = unit.strip(string.digits)
    df['Unit'] = unit
    df['Data Source'] = 'BP'
    df['Spatial Scale'] = 'National'
    for country in df.index:
        strings = ['Total', 'Rest of World', 'Other', 'European Union', 'OECD', 'Central America', \
                   'Eastern Africa', 'Middle Africa', 'Western Africa', 'OPEC']
        for s in strings:
            if s in country:
                df.drop(country, inplace=True)
    iso2 = []
    for country in df.index:
        if country=='USSR':
            iso2.append('SU')
        elif country=='Netherlands Antilles':
            iso2.append('AN')
        else:
            iso2.append(coco.convert(names=country, to='iso2'))
    df['Country Code'] = iso2
    df.reset_index(drop=True, inplace=True)
    df.replace({'Petajoules':'petajoules','Million tonnes':'million metric tons',
                'Million tonnes ':'million metric tons','million tonnes ':'million metric tons',
                'Thousand tonnes':'thousand metric tons','Tonnes':'metric tons',
                'Terawatt-hours':'TWh','Billion cubic metres':'billion cubic metres',
               'Thousand barrels daily':'thousand barrels/day'}, inplace=True)
    return df

In [20]:
bp_biofuels = read_bp2('Biofuels production - PJ')
bp_biofuels['Metric'] = 'Annual Production'
bp_biofuels = tech_name(bp_biofuels, 'Biofuels Production')
bp.append(bp_biofuels)

In [21]:
bp = pd.concat(bp)

In [22]:
omit = []
for col in bp.columns:
    if type(col) != int:
        omit.append(col)
empty_rows = bp.drop(columns=omit)
empty_rows.dropna(how='all', inplace=True)
na_idx = []
for country in bp.index:
    if country not in empty_rows.index:
        na_idx.append(country)
bp.drop(na_idx, inplace=True)

In [23]:
bp.to_csv('cleaned data/bp.csv')